In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!sudo apt install tesseract-ocr-tha
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr-tha is already the newest version (4.00~git24-0e00fe6-1.2).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [2]:
import cv2
import numpy as np
import pytesseract
from PIL import ImageFont, ImageDraw,Image
from google.colab.patches import cv2_imshow

In [3]:
def box_cut(values):
    row_img=[]
    dtype = [('x', int), ('y', int), ('w', int), ('h', int)]
    box_array = np.array(values, dtype = dtype)
    sort_box_array = np.sort(box_array, order = 'x')
    for position in sort_box_array:
        x, y, w, h = position
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
        
        #cv2_imshow(img)
        bg_img = np.zeros((img.shape[0], img.shape[1]))
        bg_img[y:y+h, x:x+w] = 1
        img_cut = img_copy[y:y+h, x:x+w,:]

        #cv2_imshow(img_cut)
        row_img.append(img_cut)
        #cv2.waitKey(1000)

    return row_img

pos_con = []
bitw_img = []

keep_day=[[],[],[],[],[]]

keep_box=[[],[],[],[],[]]


#read an image
img = cv2.imread("/content/drive/MyDrive/image_project/table1.jpg")

#resize image(optional)
"""scale_percent = 100
width = int(img.shape[1] * scale_percent / 100)
hight = int(img.shape[0] * scale_percent / 100)
dim = (width, hight)
img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)"""

img_copy = img.copy()
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
threshold_img = cv2.adaptiveThreshold(gray_img, 255,
                                      cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,29,0)
dilation = cv2.dilate(threshold_img, (2, 2), 10)
contours, hierarchy = cv2.findContours(dilation, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
contours = np.flip(contours, 0)
count = 0

if (len(contours) > 0):
    for i, con in enumerate(contours):
        areas_con = cv2.contourArea(con)
        x, y, w, h = cv2.boundingRect(con)
        if (areas_con < 19000 and areas_con > 4000):
                #print("index : {0} ==> area_con = {1}".format(i, areas_con))
                if (y>68) and (y<156):
                    keep_day[0].append(x)
                    keep_box[0].append((x, y, w, h))
                elif (y>156) and (y<244):
                    keep_day[1].append(x)
                    keep_box[1].append((x, y, w, h))
                elif (y>244) and (y<332):
                    keep_day[2].append(x)
                    keep_box[2].append((x, y, w, h))
                elif (y>332) and (y<420):
                    keep_day[3].append(x)
                    keep_box[3].append((x, y, w, h))
                elif (y>420) and (y<508):
                    keep_day[4].append(x)
                    keep_box[4].append((x, y, w, h))
                count += 1

    for row_day in keep_box:
          pos_con.append(box_cut(row_day))
    img_append = np.array(pos_con)

print("number of contours in condition is : {}".format(count))
cv2.waitKey(0)
cv2.destroyAllWindows()


number of contours in condition is : 50


/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:232: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  m = asarray(m)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [43]:
def extract(image):
    #closing
    kernel = np.ones((2,2),np.uint8)
    closing = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    # Apply OCR on the image
    costom_config=r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(closing, lang='tha', config= costom_config)
    return str(text)

In [89]:
def filter_str(str1, f_list):
    for i in f_list:
        str1 = str1.replace(i,'')
    str1 = str1.replace("ส/17",'4/17').replace("6/17",'4/17').replace('4/17',' 4/17').replace('๓','พุธ')
    while "  " in str1:
        str1 = str1.replace("  ",' ')
    ################################
    str_list = str1.split()
    for cha in str_list:
        if len(cha)<3:
            str_list.remove(cha)
    str1 = " ".join(str_list)
    return str1

In [90]:
pytesseract.pytesseract.tesseract_cmd = ( r'/usr/bin/tesseract' )

table=[]
for j,row in enumerate(img_append):
    row_= ['','','','','','','','','','','','']
    i = 0
    for img in row:
        text=filter_str(extract(img).replace('\n',' '),['|','-','.',',',':','(',')','๒'])
        row_[i]=text
        i += 1
        if img.shape[1]>110: 
            row_[i]=text
            i += 1
    table.append(row_)

In [100]:
output={}
for row in table:
    day={}
    for i,ele in enumerate(row):
        if i != 0:
            ele=ele.split()
            if len(ele)>=2:
                priod={"subject":ele[0],'room':ele[-1]}
            else:priod={"subject":None}
            day[i]=priod
    output[row[0]] = day


In [108]:
import json
#json_string = json.dumps(output).encode('utf8')
out_file = open("output.json", "w") 
json.dump(output, out_file, indent = 6,  ensure_ascii=False ) 
out_file.close() 